In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import os
import time

# Configurar rutas
geckodriver_path = r"C:\geckodriver-v0.36.0-win64\geckodriver.exe"
firefox_binary_path = r"C:\Program Files\Mozilla Firefox\firefox.exe"

# Opciones de Selenium
options = FirefoxOptions()
options.binary_location = firefox_binary_path
options.add_argument("--headless")  # Oculta la ventana del navegador

service = FirefoxService(executable_path=geckodriver_path)
driver = webdriver.Firefox(service=service, options=options)

# URL principal
driver.get("https://www.boticasysalud.com/tienda/catalogo/medicinasytratamientos")
time.sleep(5)

# Scroll para cargar todos los productos
delay = 2
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")
retries = 0
max_retries = 5

while retries < max_retries:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        retries += 1
    else:
        retries = 0  # reset if page keeps loading

    last_height = new_height


# Esperar que cargue todo
time.sleep(3)

# Obtener tarjetas de producto
tarjetas = driver.find_elements(By.CSS_SELECTOR, "div.product-card__information")
print(f"🧪 Tarjetas encontradas: {len(tarjetas)}")

nombres = []
precios = []
principios = []

for tarjeta in tarjetas:
    try:
        nombre = tarjeta.find_element(By.CSS_SELECTOR, "div.product-card__name").text.strip()
    except NoSuchElementException:
        nombre = "No encontrado"

    try:
        precio = tarjeta.find_element(By.CSS_SELECTOR, "div.product__summary-new-price").text.strip().replace("S/", "").replace(",", ".")
        precio = float(precio)
    except Exception:
        continue  # Si no hay precio, no guardar nada

    try:
        principio = tarjeta.find_element(By.XPATH, ".//div[contains(text(), 'mg')]").text.strip()
    except NoSuchElementException:
        principio = "No encontrado"

    nombres.append(nombre)
    precios.append(precio)
    principios.append(principio)

# Cerrar navegador
driver.quit()

# Crear DataFrame
if nombres:
    df = pd.DataFrame({
        "Producto": nombres,
        "Precio": precios,
        "PrincipioActivo": principios
    })

    # Guardar CSV
    output_path = "../data/raw/boticasysalud_raw_data.csv"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False, encoding="utf-8")
    print(f"✅ Archivo guardado: {output_path}")
else:
    print("⚠️ No se extrajo ningún producto. Revisa los selectores o estructura del sitio.")


🧪 Tarjetas encontradas: 2755
✅ Archivo guardado: ../data/raw/boticasysalud_raw_data.csv
